In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA

import itertools

import warnings
warnings.filterwarnings("ignore")

In [20]:
pjme_data = pd.read_csv('my_PJME_MW.csv', index_col = [0], parse_dates=[0])
pjme_data.head(2)

,Time,PJME_MW,year,month,dayofmonth,quarter,dayofyear,weekofyear,dayofweek,hour,is_holiday,is_prev_holiday,is_next_holiday
Date,,,,,,,,,,,,,
2002-01-01,01:00:00,30393.0,2002,1,1,1,1,1,1,1,1,0,0
2002-01-01,02:00:00,29265.0,2002,1,1,1,1,1,1,2,1,0,0


In [21]:
# ресемплинг по месяцам
pjme_data = pjme_data.resample("M").sum().head(10)
pjme_data.head(5)

,PJME_MW,year,month,dayofmonth,quarter,dayofyear,weekofyear,dayofweek,hour,is_holiday,is_prev_holiday,is_next_holiday
Date,,,,,,,,,,,,
2002-01-31,23089022.0,1487486,743,11903,743,11903,2135,2159,8556,47,24,48
2002-02-28,20320720.0,1345344,1344,9744,672,30576,4752,2016,7728,24,24,24
2002-03-31,21483191.0,1489488,2232,11904,744,55800,8376,2376,8556,0,0,0
2002-04-30,20516472.0,1439438,2876,11153,1438,75863,11266,2034,8277,0,0,0
2002-05-31,20886798.0,1489488,3720,11904,1488,101184,14880,2232,8556,24,24,24


In [54]:
# Определим p, d и q 
q = 2
p = 4
d = 1
pdq = [p, d, q]

N2=30
warnings.filterwarnings("ignore") # отключает предупреждения

modb = sm.tsa.statespace.SARIMAX(pjme_data['PJME_MW'].values,
                               order=pdq,seasonal_order=[0,0,0,0], trend='ct',
                               enforce_stationarity=False,
                               enforce_invertibility=False)
resultsb = modb.fit()
print('ARIMA{} - AIC:{}'.format(pdq, resultsb.aic))

print(resultsb.summary().tables[0],resultsb.summary().tables[1])

ARIMA[4, 1, 2] - AIC:229037171.9857456
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                   10
Model:               SARIMAX(4, 1, 2)   Log Likelihood          -114518576.993
Date:                Tue, 03 Nov 2020   AIC                      229037171.986
Time:                        06:01:28   BIC                      229037168.471
Sample:                             0   HQIC                     229037162.552
                                 - 10                                         
Covariance Type:                  opg                                         
============================================================================== ==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    

In [57]:
brent_data = pd.read_excel('нефть-brent 2010-2019.xlsx', index_col=[0], parse_dates=[0])
brent_data.head(5)

,Значение
Дата,
2019-09-01,58.84
2019-08-30,59.19
2019-08-29,60.37
2019-08-28,59.84
2019-08-27,59.59


In [58]:
brent_data = brent_data.resample("7D").sum().head(10)
brent_data.head(5)

,Значение
Дата,
2010-09-01,537.21
2010-09-08,470.20
2010-09-15,471.33
2010-09-22,470.64
2010-09-29,498.23


In [72]:
N = 40
nforecast = N  # полное число предиктов (тест+ чистый предикт)

model = ARIMA(brent_data[:-N], order=(2,2,2))
model_fit = model.fit(disp=False)

pred_dynamic = model_fit.forecast(nforecast)    #делаем предсказание на 30 точtк от текущего конца ряда
pred_dynamic_opt = results_opt.forecast(nforecast)

y_ps = results_opt.predict(len(brent_data[:-N]), len(brent_data[:-N]), typ='levels')
y_fs = results_opt.forecast(2*N )

plt.figure(figsize=(16,7))
plt.plot(brent_data[-N*2:],'r',label ='true')
plt.plot(N,y_p,'*b',label ='model')
plt.plot(N,y_ps,'*b',label ='model sarimax')
plt.plot(np.arange(2*N)+N,y_fs,'g',label ='predict sarimax')
plt.plot(np.arange(2*N)+N,y_f[0],'b',label ='predict')
plt.plot(np.arange(2*N)+N,y_f[2][:,0],'--b', label = 'граница')
plt.plot(np.arange(2*N)+N,y_f[2][:,1],'--b')
plt.xlabel('x')
plt.ylabel('brent_data')
plt.title('Предсказание последних N точек ряда  и следующих N точек ряда')
plt.legend()
plt.show()

ValueError: Insufficient degrees of freedom to estimate